In [2]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import os

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
cyberball_raw_dir = Path('Physiological (ECG, EDA, RSP)/')
cyberball_interim_dir = Path('interim/')

In [6]:
for txt in tqdm(list(cyberball_raw_dir.glob("*.txt"))):
    # print(txt)
    try:
        _, _, patient_id, paradigm, signal = txt.name.split(".")[0].split("_")
        df_sig = pd.read_csv(
            txt, skiprows=3, sep=" ", names=["timestamp", signal, "trigger", "dunno"]
        )
        t_start = pd.to_datetime(open(txt).readlines()[1].strip()).tz_localize(
            "Europe/Brussels"
        )
        df_sig.timestamp = t_start + pd.to_timedelta(df_sig.timestamp, unit='ms')
        df_sig = df_sig.set_index('timestamp').iloc[:, :2]
        df_sig[signal] = df_sig[signal].astype(np.float32)
        # df_sig["trigger"] = df_sig["trigger"].astype(np.int32)
        save_dir = cyberball_interim_dir.joinpath(patient_id)
        if not save_dir.is_dir():
            os.makedirs(save_dir)
        df_sig.to_parquet(save_dir.joinpath(f'{signal}_{paradigm}.parquet'))
    except Exception as e:
        print("[E]", txt.name, e)
    # /1024_ppt_8_cybb_SCL.txt

0it [00:00, ?it/s]

In [47]:
from plotly_resampler import FigureWidgetResampler, FigureResampler, EfficientLTTB
import plotly.graph_objects as go

In [48]:
from plotly.subplots import make_subplots

In [49]:
import numpy as np

In [66]:
%%timeit
int(s_i[0])

275 ns ± 9.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [28]:
from datetime import datetime
import lttbc
from plotly_resampler import EfficientLTTB

elttb = EfficientLTTB(interleave_gaps=False)

In [55]:
n = 50_000_000
s = pd.Series(index=pd.date_range(datetime.now(), freq='1.23ms', periods=n), data=np.arange(n))

s_v = s.cat.codes.values if str(s.dtype) == "category" else s.values
s_i = s.index.values

In [91]:
%%timeit # the current (incorrect implementation) - 5M
elttb._aggregate(s, 1000)

18.1 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
%%timeit # the current (incorrect implementation) - 50M
elttb._aggregate(s, 1000)

115 ms ± 6.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

In [54]:
%%timeit # the current (incorrect implementation) - 50M
elttb._aggregate(s, 1000)

18.1 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%%timeit # the current (incorrect implementation) - 50M
elttb._aggregate(s, 1000)

106 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
s_i

array(['2022-05-19T18:23:50.506970000', '2022-05-19T18:23:50.508200000',
       '2022-05-19T18:23:50.509430000', ...,
       '2022-05-19T20:06:20.503280000', '2022-05-19T20:06:20.504510000',
       '2022-05-19T20:06:20.505740000'], dtype='datetime64[ns]')

In [ ]:
s_i0 = s_i[0].astype(np.int64)
idx, data = lttbc.downsample(
    (s_i.astype(np.int64) - s_i0).astype(np.float64), s_v, 1000
)
idx = pd.to_datetime(idx.astype(int) + s_i0, unit="ns", utc=True).tz_convert(s.index.tz)
idx.values

In [40]:
(s_i.astype(np.int64) - s_i0).astype(float)

array([0.000000e+00, 1.000000e+06, 2.000000e+06, ..., 4.999997e+12,
       4.999998e+12, 4.999999e+12])

In [30]:
idx

DatetimeIndex(['1969-12-31 23:59:59.517391344',
               '1969-12-31 23:59:57.370424048',
               '1970-01-01 00:00:01.664424048',
               '1969-12-31 23:59:57.370489456',
               '1970-01-01 00:00:01.664489456',
               '1969-12-31 23:59:57.370587568',
               '1970-01-01 00:00:01.664587568',
               '1969-12-31 23:59:57.370652976',
               '1970-01-01 00:00:01.664652976',
               '1969-12-31 23:59:57.370718384',
               ...
               '1970-01-01 00:00:01.664131760',
               '1969-12-31 23:59:57.370197168',
               '1970-01-01 00:00:01.664197168',
               '1969-12-31 23:59:57.370262576',
               '1970-01-01 00:00:01.664262576',
               '1969-12-31 23:59:57.370327984',
               '1970-01-01 00:00:01.664360688',
               '1969-12-31 23:59:57.370426096',
               '1970-01-01 00:00:01.664426096',
               '1970-01-01 00:00:00.174458800'],
              dtype=

In [20]:
elttb._aggregate(s, 1000)

2022-05-19 18:16:44.181464064           0
2022-05-19 18:16:47.180464128        2999
2022-05-19 18:17:36.181464064       52000
2022-05-19 18:19:14.181464064      150000
2022-05-19 18:20:05.180464128      200999
                                   ...   
2022-05-20 08:06:42.181464064    49798000
2022-05-20 08:07:33.180464128    49848999
2022-05-20 08:09:11.180464128    49946999
2022-05-20 08:09:12.181464064    49948000
2022-05-20 08:10:04.180464128    49999999
Name: None, Length: 1000, dtype: int64

In [84]:
s_v = s.cat.codes.values if str(s.dtype) == "category" else s.values
s_i = s.index.values

In [85]:
%%timeit
s_i0 = int(s_i[0])
idx, data = lttbc.downsample(s_i.astype(int) - s_i0, s_v, 1000)
idx = pd.to_datetime(
    idx.astype(int) + s_i0, unit="ns", utc=True
).tz_convert(s.index.tz)

54.8 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
%%timeit
idx, data = lttbc.downsample(s_i.astype(int), s_v, 1000)
idx = pd.to_datetime(idx, unit="ns", utc=True).tz_convert(s.index.tz)

47.5 ms ± 7.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


TODO here tests for plotly resampler

In [65]:
s_if = s_i.astype(int) # casting='safe')
(s_if - s_if[0]).astype(np.float64).astype(int) + s_if[0]

array([1629787951550000000, 1629787951650000000, 1629787951750000000, ...,
       1629795689350000000, 1629795689450000000, 1629795689550000000])

In [41]:
import lttbc

In [47]:
idx, data = lttbc.downsample(s_i, df_scl["SCL"].values, 1000)

In [48]:
pd.to_datetime(idx.astype(int))

DatetimeIndex(['2021-08-24 06:52:31.550000128',
               '2021-08-24 06:52:34.249999872',
               '2021-08-24 06:52:39.350000128',
               '2021-08-24 06:52:47.150000128',
               '2021-08-24 06:53:02.550000128',
               '2021-08-24 06:53:02.649999872',
               '2021-08-24 06:53:10.750000128',
               '2021-08-24 06:53:18.150000128',
               '2021-08-24 06:53:30.350000128',
               '2021-08-24 06:53:33.649999872',
               ...
               '2021-08-24 09:00:19.750000128',
               '2021-08-24 09:00:27.449999872',
               '2021-08-24 09:00:35.249999872',
               '2021-08-24 09:00:42.950000128',
               '2021-08-24 09:00:50.750000128',
               '2021-08-24 09:00:58.449999872',
               '2021-08-24 09:01:06.249999872',
               '2021-08-24 09:01:13.950000128',
               '2021-08-24 09:01:21.750000128',
               '2021-08-24 09:01:29.550000128'],
              dtype=

In [36]:
type(s_i[0])

numpy.int64

In [34]:
type(idx[0])

numpy.float64

In [28]:
%%timeit
pd.RangeIndex(start=0, stop=50_000_000).values

156 ms ± 72.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
df_scl.index[[1, 2, 3]]

DatetimeIndex(['2021-08-24 08:52:31.650000+02:00',
               '2021-08-24 08:52:31.750000+02:00',
               '2021-08-24 08:52:31.850000+02:00'],
              dtype='datetime64[ns, Europe/Brussels]', name='timestamp', freq=None)

In [17]:
pd.to_datetime(s_i.astype(int)

array([1629787951550000000, 1629787951650000000, 1629787951750000000, ...,
       1629795689350000000, 1629795689450000000, 1629795689550000000])

In [ ]:
fwr = FigureResampler(
    # make_subplots(rows=2, cols=1, shared_xaxes=True),
    make_subplots(specs=[[{"secondary_y": True}]]),
)
fwr.add_trace(go.Scattergl(name="scl"), hf_x=df_scl.index, hf_y=df_scl["SCL"])
fwr.add_trace(
    go.Scattergl(name="trigger"),
    hf_x=df_scl.index,
    hf_y=(df_scl["trigger"] != -9999).astype(int),
    downsampler=EfficientLTTB(interleave_gaps=False),
    hf_hovertext=df_scl["trigger"],
    secondary_y=True,
)
fwr.update_layout(template="plotly_white", height=500, showlegend=True)
fwr.show_dash(mode="inline")